In [2]:
!pip install -qU langchain-groq langchain-community chromadb sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 9.9 MB/s eta 0:00

In [3]:
import os
from google.colab import userdata

# This line looks for the secret you named 'GROQ_API_KEY'
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

print("Key is set! You can now proceed to the RAG pipeline.")


Key is set! You can now proceed to the RAG pipeline.


In [4]:
import os
os.makedirs('policies', exist_ok=True)
policies = {
    "refund_policy.txt": """
    REFUND POLICY
    1. Window: Full refunds are available within 30 days of purchase.
    2. Software Exception: Digital software licenses are strictly non-refundable once the activation key has been displayed to the user.
    3. Condition: Physical items must be returned in 'Like New' condition. A 15% restocking fee applies if the original seal is broken.
    4. Method: All refunds are issued to the original payment method. If the original method is unavailable, store credit will be issued.
    """,

    "shipping_policy.txt": """
    SHIPPING & DELIVERY
    1. Regions: We currently only ship to the United States (all 50 states) and Canada. We do not offer international shipping to Europe or Asia.
    2. Costs: Standard shipping (5-7 days) is free for orders over $50. Otherwise, a flat rate of $5.99 applies.
    3. Express Shipping: 2-day priority shipping is available for $15, regardless of order total.
    4. P.O. Boxes: We cannot deliver Express Shipping orders to P.O. Boxes; a physical street address is required.
    """,

    "cancellation_policy.txt": """
    SUBSCRIPTION & CANCELLATION
    1. Self-Service: Users can cancel subscriptions anytime via the Account Dashboard.
    2. Billing Cycles: Cancellations take effect at the end of the current 30-day billing cycle. No pro-rated refunds are provided for partial months.
    3. Late Fee: A $10 late fee is applied to accounts that remain unpaid 5 days after the billing date.
    4. Account Deletion: Canceling a subscription does not delete your data. Data is purged only after 12 months of inactivity.
    """
}
for filename, content in policies.items():
    with open(f"policies/{filename}", "w") as f:
        f.write(content)

print("Policy documents created in /policies folder.")

Policy documents created in /policies folder.


In [18]:
import os
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_classic.chains import RetrievalQA
loader = DirectoryLoader('./policies/', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100)
splits = text_splitter.split_documents(docs)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
llm = ChatGroq(
    model_name="openai/gpt-oss-120b",
    temperature=0,
    reasoning_effort="medium",  # Special 120B parameter for deeper logic
    reasoning_format="parsed"    # Keeps reasoning tokens separate from the answer
)
template = """
You are a Senior Customer Support Analyst. Your goal is to answer questions based strictly on the provided company documents.

[STRICT GUIDELINES]
1. ONLY use information found in the Context.
2. If the context does not contain the answer, strictly respond: "I'm sorry, I cannot find information regarding that in our current policies."
3. If the answer is partially available, provide what you found and note what is missing.
4. Format your answer with clear headings and bullet points where applicable.
5. CITATION: Start your answer by stating which policy document you are citing.

[CONTEXT]
{context}

[QUESTION]
{question}

Structured Answer:"""

custom_prompt = PromptTemplate(template=template, input_variables=["context", "question"])

# 5. Create the Chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs={"prompt": custom_prompt}
)

In [20]:
import json

# Define the Evaluation Set
test_suite = [
    {"type": "Answerable", "q": "What is the restocking fee for physical items?"},
    {"type": "Answerable", "q": "Is shipping free for a $60 order?"},
    {"type": "Partially Answerable", "q": "Do you ship to Mexico and Canada?"},
    {"type": "Unanswerable", "q": "What is the customer support phone number?"},
    {"type": "Edge Case", "q": "Can I get a refund if I viewed my software license key yesterday?"}
]

print(f"{'TYPE':<22} | {'QUERY':<50} | {'EVAL'}")
print("-" * 90)

for test in test_suite:
    response = rag_chain.invoke(test["q"])
    ans = response["result"]

    # Logic for grading (in your README, you'd fill the grading manually)
    print(f"{test['type']:<22} | {test['q']:<50}")
    print(f"OUTPUT: {ans}")
    print("-" * 90)

TYPE                   | QUERY                                              | EVAL
------------------------------------------------------------------------------------------
Answerable             | What is the restocking fee for physical items?    
OUTPUT: **Cited Document:** REFUND POLICY  

**Answer:**  
- The restocking fee for physical items is **15%** if the original seal is broken.
------------------------------------------------------------------------------------------
Answerable             | Is shipping free for a $60 order?                 
OUTPUT: **Cited Policy Document:** Shipping & Delivery (Regions, Costs, Express Shipping, P.O. Boxes)

**Answer:**

- **Standard Shipping:**  
  - Free for orders **over $50**.  
  - A $60 order meets this criterion, so standard shipping (5‑7 days) is free.

- **Express Shipping:**  
  - Costs **$15** regardless of order total.  
  - If you choose express shipping, the fee applies even for a $60 order.

**Summary:** Yes, shipping is free

In [21]:
with open('requirements.txt', 'w') as f:
    f.write("langchain-groq\nlangchain-community\nlangchain-classic\nchromadb\nsentence-transformers\n")